In [2]:
from typing import Set
import re

def tokenize(text):
    text = text.lower()
    all_words = re.findall("[a-z0-9']+", text)
    return set(all_words)

tokenize("Banana apPle Orange APPle")

{'apple', 'banana', 'orange'}

In [32]:
from sklearn.datasets import fetch_20newsgroups
from sklearn import datasets
import pandas as pd

categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
train_data = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
test_data = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
len(train_data.data),len(test_data.data)
print(len(test_data.data))
print(len(train_data.data))

dataset = fetch_20newsgroups(subset='all', categories=categories)
print(dataset.target)

print("%d documents" % len(dataset.data)) # Nombre de documents
print("%d categories" % len(dataset.target_names)) # Nombre de categories
print("contenu de l'attribut target: %a" % dataset.target) # Liste des categories de chaque document
print("contenu de l'attribut target_names: %a" % dataset.target_names) # Liste des categories

print(dataset.data[0]) # Imprime le 1er texte

print(dataset.target[0]) 

def tokenize(text):
    text = text.lower()
    all_words = re.findall("[a-z0-9']+", text)
    return set(all_words)

tokenize(dataset.data[0])

print(len(dataset.data))
print(test_data.data[0])


1502
2257
3759 documents
4 categories
contenu de l'attribut target: array([2, 3, 3, ..., 2, 0, 2])
contenu de l'attribut target_names: ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']
From: geb@cs.pitt.edu (Gordon Banks)
Subject: Re: "CAN'T BREATHE"
Article-I.D.: pitt.19440
Reply-To: geb@cs.pitt.edu (Gordon Banks)
Organization: Univ. of Pittsburgh Computer Science
Lines: 23

In article <1993Mar29.204003.26952@tijc02.uucp> pjs269@tijc02.uucp (Paul Schmidt) writes:
>I think it is important to verify all procedures with proper studies to
>show their worthiness and risk.  I just read an interesting tidbit that 
>80% of the medical treatments are unproven and not based on scientific 
>fact.  For example, many treatments of prostate cancer are unproven and
>the treatment may be more dangerous than the disease (according to the
>article I read.)

Where did you read this?  I don't think this is true.  I think most
medical treatments are based on science, although it is dif

In [46]:
from typing import List, Tuple, Dict, Iterable
import math
from collections import defaultdict

class NaiveBayesClassifier:
    def __init__(self, k = 0.5):
        self.k = k
        self.tokens = set()
        self.token_atheism_counts = {}
        self.token_graphics_counts = {}
        self.token_med_counts = {}
        self.token_christian_counts = {}
        self.atheism_messages = self.graphics_messages = self.med_messages = self.christian_messages = 0

    def train(self, dataset):
        for i in range(len(dataset.data)):
            if dataset.target[i] == 0:
                self.atheism_messages += 1
            elif dataset.target[i] == 1:
                self.graphics_messages += 1
            elif dataset.target[i] == 2:
                self.med_messages += 1
            elif dataset.target[i] == 3:
                self.christian_messages += 1

            for token in tokenize(dataset.data[i]):
                self.tokens.add(token)
                if dataset.target[i] == 0 and token not in self.token_atheism_counts:
                 self.token_atheism_counts[token] = 0
                elif dataset.target[i] == 0 and token in self.token_atheism_counts:
                 self.token_atheism_counts[token] += 1
                elif dataset.target[i] == 1 and token not in self.token_graphics_counts:
                  self.token_graphics_counts[token] = 0
                elif dataset.target[i] == 1 and token in self.token_graphics_counts:
                  self.token_graphics_counts[token] += 1
                elif dataset.target[i] == 2 and token not in self.token_med_counts:
                  self.token_med_counts[token] = 0
                elif dataset.target[i] == 2 and token in self.token_med_counts:
                  self.token_med_counts[token] += 1
                elif dataset.target[i] == 3 and token not in self.token_christian_counts:
                  self.token_christian_counts[token] = 0
                elif dataset.target[i] == 3 and token in self.token_christian_counts:
                  self.token_christian_counts[token] += 1

    def _probabilities(self, token):
        if token in self.token_atheism_counts:
            atheism = self.token_atheism_counts[token]
        else:
           atheism = 0
        if token in self.token_graphics_counts:
            graphics = self.token_graphics_counts[token]
        else:
           graphics = 0
        if token in self.token_med_counts:
            med = self.token_med_counts[token]
        else:
           med = 0
        if token in self.token_christian_counts:
            christian = self.token_christian_counts[token]
        else:
           christian = 0

        p_token_atheism = (atheism + self.k) / (self.atheism_messages + 2 * self.k)
        p_token_graphics = (graphics + self.k) / (self.graphics_messages + 2 * self.k)
        p_token_med = (med + self.k) / (self.med_messages + 2 * self.k)
        p_token_christian = (christian + self.k) / (self.christian_messages + 2 * self.k)
        return p_token_atheism, p_token_graphics, p_token_med, p_token_christian
    
    def predict(self, text):
        target=[]
        for i in range(len(text.data)):
           target.append(text.target[i])
        print(target)
        
        list_of_classes = []
        for i in range(len(text.data)):
            text_tokens = tokenize(text.data[i])
            log_prob_if_atheism = log_prob_if_graphics = log_prob_if_med = log_prob_if_christian = 0.0

            for token in self.tokens:
                prob_if_atheism, prob_if_graphics, prob_if_med, prob_if_christian = self._probabilities(token)

                if token in text_tokens:
                    log_prob_if_atheism += math.log(prob_if_atheism)
                    log_prob_if_graphics += math.log(prob_if_graphics)
                    log_prob_if_med += math.log(prob_if_med)
                    log_prob_if_christian += math.log(prob_if_christian)
            
                else:
                    log_prob_if_atheism += math.log(1.0 - prob_if_atheism)
                    log_prob_if_graphics += math.log(1.0 - prob_if_graphics)
                    log_prob_if_med += math.log(1.0 - prob_if_med)
                    log_prob_if_christian += math.log(1.0 - prob_if_christian)

            prob_if_atheism = math.exp(log_prob_if_atheism)
            prob_if_graphics = math.exp(log_prob_if_graphics)
            prob_if_med = math.exp(log_prob_if_med)
            prob_if_christian = math.exp(log_prob_if_christian)

            if (prob_if_atheism+prob_if_graphics+prob_if_med+prob_if_christian) == 0.0:
                end_prob_if_atheism =  end_prob_if_graphics = end_prob_if_med = end_prob_if_christian = 0
            else: 
                end_prob_if_atheism = prob_if_atheism/(prob_if_atheism+prob_if_graphics+prob_if_med+prob_if_christian)
                end_prob_if_graphics = prob_if_graphics/(prob_if_atheism+prob_if_graphics+prob_if_med+prob_if_christian)
                end_prob_if_med = prob_if_med/(prob_if_atheism+prob_if_graphics+prob_if_med+prob_if_christian)
                end_prob_if_christian = prob_if_christian/(prob_if_atheism+prob_if_graphics+prob_if_med+prob_if_christian)

            if max(end_prob_if_atheism,end_prob_if_graphics,end_prob_if_med,end_prob_if_christian)==end_prob_if_atheism:
                list_of_classes.append(0)
            elif max(end_prob_if_atheism,end_prob_if_graphics,end_prob_if_med,end_prob_if_christian)==end_prob_if_graphics:
                list_of_classes.append(1)
            elif max(end_prob_if_atheism,end_prob_if_graphics,end_prob_if_med,end_prob_if_christian)==end_prob_if_med:
                list_of_classes.append(2)
            elif max(end_prob_if_atheism,end_prob_if_graphics,end_prob_if_med,end_prob_if_christian)==end_prob_if_christian:
                list_of_classes.append(3)
        print(len(list_of_classes))
        return list_of_classes
        
model = NaiveBayesClassifier(k=0.5)
print(model.train(train_data))
print(len(model.tokens))
print(model.christian_messages)
print(model.med_messages)
print(model.graphics_messages)
print(model.atheism_messages)
print(model.token_atheism_counts)
print(model.token_graphics_counts)
print(model.token_med_counts)
print(model.token_christian_counts)
print(model.predict(test_data))




None
36374
599
594
584
480
{'emergence': 0, 'independently': 7, 'lot': 30, 'free': 50, 'atheism': 91, 'a': 438, 'note': 34, 'unicorns': 9, 'without': 70, 'your': 232, 'power': 25, 'bible': 63, 'perfect': 19, 'good': 94, 'there': 226, 'nothing': 75, '091139': 1, 'another': 69, 'about': 223, 'gives': 9, 'omnipotent': 7, 'catholic': 13, 'even': 136, 'think': 176, 'absence': 9, 'am': 91, 'no': 219, 'pet': 1, 'argument': 59, "didn't": 57, 'but': 299, 'similar': 36, 'defined': 30, 'like': 154, "don't": 215, 'therefore': 63, 'subject': 479, 'error': 16, 'an': 253, 'disagree': 12, 'allows': 10, 'i': 407, 'when': 138, 'foot': 7, 'created': 34, 'moral': 69, 'nature': 38, 'first': 82, 'view': 42, 'already': 26, "i'm": 129, 'well': 140, 'must': 117, 'ubiquitous': 0, 'which': 180, 'letting': 6, 'people': 194, 'were': 150, 'definition': 40, 'something': 116, 'claims': 28, 'being': 137, 'out': 154, 'weak': 25, 'one': 248, 'conceivable': 2, 'lines': 479, "being's": 0, '114': 0, 'fixed': 1, 'shows': 14